In [6]:
import pandas as pandas
dataframe = pandas.read_csv('India_Menu.csv')
print(dataframe.head(5))

  Menu Category              Menu Items Per Serve Size  Energy (kCal)  \
0  Regular Menu        McVeggie™ Burger          168 g         402.05   
1  Regular Menu    McAloo Tikki Burger®          146 g         339.52   
2  Regular Menu  McSpicy™ Paneer Burger          199 g         652.76   
3  Regular Menu       Spicy Paneer Wrap          250 g         674.68   
4  Regular Menu     American Veg Burger          177 g         512.17   

   Protein (g)  Total fat (g)  Sat Fat (g)  Trans fat (g)  Cholesterols (mg)  \
0        10.24          13.83         5.34           0.16               2.49   
1         8.50          11.31         4.27           0.20               1.47   
2        20.29          39.45        17.12           0.18              21.85   
3        20.96          39.10        19.73           0.26              40.93   
4        15.30          23.45        10.51           0.17              25.24   

   Total carbohydrate (g)  Total Sugars (g)  Added Sugars (g)  Sodium (mg)  
0  